# VANI

**Initial notebook configuration**

In [1]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

In [2]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

/usr/workspace/iopp/.conda/envs/jupyter/bin/python
3.9.7 (default, Sep 16 2021, 16:03:39) 
[GCC 7.3.0]
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)


In [3]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Import libraries**

In [4]:
# Import VANI library
import dask
import numpy as np
import pandas as pd
from vani.vani.analyzer import Analyzer, ClusterOptions, ClusterType

**Initialize VANI Analyzer**

In [5]:
%autoreload 2

# Initialize analyzer
co = ClusterOptions(cluster_type=ClusterType.Local)
vn = Analyzer(n_workers=8, cluster_options=co, debug=True)

  0%|          | 0/3 [00:00<?, ?it/s]

Dashboard link: http://127.0.0.1:8787/status


**Analyze logs**

In [28]:
%autoreload 2

# Analysis configuration
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/hacc/nodes-32/workflow-0"
log_dir = "/p/gpfs1/iopp/parquet_app_logs/cm1/nodes-32/workflow-4"
# log_dir = "/p/gpfs1/iopp/parquet_app_logs/lbann-cosmoflow/nodes-32"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage_pegasus/nodes-32/_parquet"
# log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

# Do the analysis  
io_df_read_write, job_time = vn.analyze_parquet_logs(log_dir)    

  0%|          | 0/7 [00:00<?, ?it/s]

Logs read (0.030877497047185898)
Job time: 668.3056030273438 seconds (0.23324604146182537)
Finding problematic sizes by bin - depth: 0
fixed_sizes_by_bin
 0.000000      20.171007
 594.049425     0.216297
 297.024712     0.215934
 74.256178      0.141096
 222.768534     0.141096
 445.537069     0.141091
 519.793247     0.140733
 148.512356     0.005216
 371.280891     0.005100
-1.000000       0.000000
dtype: float64
labeled_sizes_by_bin
 0.000000      10
 594.049425     1
 297.024712     1
 74.256178      1
 222.768534     1
 445.537069     1
 519.793247     1
 148.512356     1
 371.280891     1
-1.000000       1
dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]
labeled_sizes_by_bin.index.values
[  0.         594.04942491 297.02471246  74.25617811 222.76853434
 445.53706868 519.7932468  148.51235623 371.28089057  -1.        ]
problematic_sizes_by_bin.index.values
[0.]
problematic_sizes_by_bin flagged
0.0    10
dtype: category
Categories (10, int64): [1 < 2 < 3 <

KeyError: 1

In [43]:
# 18.72338400946723 7760
# 22.468060811360676 5960
# 11.234030405680338 4451
io_df_read_write[io_df_read_write["tbin"]==11.234030405680338].count().compute()

index        4451
rank         4451
thread_id    4451
cat          4451
tstart       4451
tend         4451
func_id      4451
level        4451
arg_count    4451
args_1       4451
args_2       4451
args_3       4451
args_4       4451
args_5       4451
args_6       4451
args_7       4451
args_8       4451
args_9       4451
args_10      4451
duration     4451
tmid         4451
filename     4451
size         4451
count        4451
bandwidth    4451
tbin         4451
dtype: int64

In [45]:
int(10/2)

5

In [5]:
%%time
io_df_read_write.columns

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 18.4 µs


Index(['index', 'rank', 'thread_id', 'cat', 'tstart', 'tend', 'func_id',
       'level', 'arg_count', 'args_1', 'args_2', 'args_3', 'args_4', 'args_5',
       'args_6', 'args_7', 'args_8', 'args_9', 'args_10', 'duration', 'tmid',
       'filename', 'size', 'count', 'bandwidth'],
      dtype='object')

In [6]:
%%time
io_df_read_write["index"].count().compute()

CPU times: user 669 ms, sys: 16.1 ms, total: 685 ms
Wall time: 1.45 s


25600

In [7]:
%%time
max_duration = io_df_read_write["duration"].max().compute()
max_duration

CPU times: user 623 ms, sys: 1.46 ms, total: 625 ms
Wall time: 1.37 s


1.5205373764038086

In [8]:
n_bins = 10

tbins, tbins_step = np.linspace(0, job_time, num=n_bins, retstep=True)
tbins

array([ 0.        ,  3.7446768 ,  7.4893536 , 11.23403041, 14.97870721,
       18.72338401, 22.46806081, 26.21273761, 29.95741442, 33.70209122])

In [9]:
tbins_step/max_duration

2.4627324918180595

In [10]:
%%time

io_df_read_write["tbin"] = 0

for tbin in tbins:
    print(tbin, tbin + tbins_step, (tbin - tbins_step)/2)
    # 0 <= tmid < 3.744676801893446
    tstart_cond = io_df_read_write["tmid"].ge(tbin)
    tend_cond = io_df_read_write["tmid"].lt(tbin + tbins_step)
    io_df_read_write["tbin"] = io_df_read_write["tbin"].mask(tstart_cond & tend_cond, tbin)

0.0 3.744676801893446 -1.872338400946723
3.744676801893446 7.489353603786892 0.0
7.489353603786892 11.234030405680338 1.872338400946723
11.234030405680338 14.978707207573784 3.744676801893446
14.978707207573784 18.72338400946723 5.617015202840169
18.72338400946723 22.46806081136068 7.489353603786893
22.468060811360676 26.21273761325412 9.361692004733616
26.21273761325412 29.957414415147568 11.234030405680336
29.957414415147568 33.702091217041016 13.10636880662706
33.702091217041016 37.44676801893446 14.978707207573784
CPU times: user 166 ms, sys: 0 ns, total: 166 ms
Wall time: 154 ms


In [11]:
%%time

len(io_df_read_write[io_df_read_write["tbin"] == 0].compute())

CPU times: user 1.03 s, sys: 6.36 ms, total: 1.04 s
Wall time: 2.15 s


0

In [12]:
%%time

for tbin in tbins:
    tbin_binned = io_df_read_write[io_df_read_write["tbin"] == tbin]
    tbin_ct = tbin_binned["index"].count().compute()
    print(tbin, tbin_ct)


0.0 0
3.744676801893446 400
7.489353603786892 2200
11.234030405680338 4451
14.978707207573784 3996
18.72338400946723 7760
22.468060811360676 5960
26.21273761325412 506
29.957414415147568 327
33.702091217041016 0
CPU times: user 10.1 s, sys: 41.9 ms, total: 10.1 s
Wall time: 20.8 s


In [13]:
%%time

tbin_size_tasks = []
tbin_bw_tasks = []
tbin_ct_tasks = []

for tbin in tbins:
    tbin_binned = io_df_read_write[io_df_read_write["tbin"] == tbin]
    tbin_size_task = tbin_binned["size"].sum()/1024.0/1024.0/1024.0
    tbin_bw_task = tbin_binned["bandwidth"].mean()/1024.0 
    tbin_ct_task = tbin_binned["index"].count()
    tbin_size_tasks.append(tbin_size_task)
    tbin_bw_tasks.append(tbin_bw_task)
    tbin_ct_tasks.append(tbin_ct_task)
    
tbin_task_results = dask.compute(tbin_size_tasks, tbin_bw_tasks, tbin_ct_tasks)

tbin_task_results

CPU times: user 6.08 s, sys: 79.9 ms, total: 6.16 s
Wall time: 7.2 s


([0.0,
  24.6875,
  138.5703125,
  271.5859375,
  241.109375,
  484.3046875,
  368.46875,
  29.5859375,
  21.6875,
  0.0],
 [nan,
  1.497905556830035,
  2.239111758181178,
  3.5624963657028332,
  3.881369141153605,
  3.342765798432728,
  2.3583242297373874,
  2.6570624776827865,
  2.094487939580025,
  nan],
 [0, 400, 2200, 4451, 3996, 7760, 5960, 506, 327, 0])

In [14]:
%%time

tbin_grpd = io_df_read_write.groupby("tbin")

tbin_grp_size_task = tbin_grpd["size"].sum()/1024.0/1024.0/1024.0
tbin_grp_bw_task = tbin_grpd["bandwidth"].mean()/1024.0
tbin_grp_ct_task = tbin_grpd["index"].count()

tbin_task_results2 = dask.compute(tbin_grp_size_task, tbin_grp_bw_task, tbin_grp_ct_task)

tbin_task_results2

CPU times: user 1.47 s, sys: 37.5 ms, total: 1.51 s
Wall time: 2.6 s


(tbin
 14.978707    241.109375
 18.723384    484.304688
 22.468061    368.468750
 26.212738     29.585938
 29.957414     21.687500
 11.234030    271.585938
 7.489354     138.570312
 3.744677      24.687500
 Name: size, dtype: float64,
 tbin
 14.978707    3.881369
 18.723384    3.342766
 22.468061    2.358324
 26.212738    2.657062
 29.957414    2.094488
 11.234030    3.562496
 7.489354     2.239112
 3.744677     1.497906
 Name: bandwidth, dtype: float64,
 tbin
 14.978707    3996
 18.723384    7760
 22.468061    5960
 26.212738     506
 29.957414     327
 11.234030    4451
 7.489354     2200
 3.744677      400
 Name: index, dtype: int64)

In [15]:
tbin_grp_size, tbin_grp_bw, tbin_grp_ct = tbin_task_results2
tbin_grp_ct.sum()

25600

In [16]:
labels = [i for i in range(1, n_bins+1)]
labels

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [17]:
tbin_grp_bw

tbin
14.978707    3.881369
18.723384    3.342766
22.468061    2.358324
26.212738    2.657062
29.957414    2.094488
11.234030    3.562496
7.489354     2.239112
3.744677     1.497906
Name: bandwidth, dtype: float64

In [18]:
tbin_grp_size_binned = pd.cut(tbin_grp_size.sort_values(ascending=False), bins=n_bins, labels=labels)
tbin_grp_size_binned

tbin
18.723384    10
22.468061     8
11.234030     6
14.978707     5
7.489354      3
26.212738     1
3.744677      1
29.957414     1
Name: size, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

In [19]:
tbin_grp_bw_binned = pd.cut(tbin_grp_bw.sort_values(ascending=True), bins=n_bins, labels=labels[::-1])
tbin_grp_bw_binned

tbin
3.744677     10
29.957414     8
7.489354      7
22.468061     7
26.212738     6
18.723384     3
11.234030     2
14.978707     1
Name: bandwidth, dtype: category
Categories (10, int64): [10 < 9 < 8 < 7 ... 4 < 3 < 2 < 1]

In [20]:
tbin_grp_ct_binned = pd.cut(tbin_grp_ct.sort_values(ascending=False), bins=n_bins, labels=labels)
tbin_grp_ct_binned

tbin
18.723384    10
22.468061     8
11.234030     6
14.978707     5
7.489354      3
26.212738     1
3.744677      1
29.957414     1
Name: index, dtype: category
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

In [21]:
tbin_grp_size_binned.astype(int).multiply(tbin_grp_bw_binned.astype(int)).sort_values(ascending=False)

tbin
22.468061    56
18.723384    30
7.489354     21
11.234030    12
3.744677     10
29.957414     8
26.212738     6
14.978707     5
dtype: int64

In [22]:
problematic_bins = tbin_grp_size_binned[tbin_grp_size_binned.eq(tbin_grp_bw_binned) == False]
problematic_bins

TypeError: Categoricals can only be compared if 'categories' are the same.

In [17]:
problematic_bins_cleared = problematic_bins[problematic_bins != "0"]
problematic_bins_cleared

tbin
22.468061    9
18.723384    9
14.978707    5
Name: size, dtype: category
Categories (10, object): ['0' < '1' < '2' < '3' ... '6' < '7' < '8' < '9']

In [18]:
pd.DataFrame(problematic_bins_cleared)

,size
tbin,
22.468061,9
18.723384,9
14.978707,5


In [ ]:
io_df_read_write_worst = io_df_read_write[io_df_read_write["tbin"].isin(problematic_bins_cleared)]

In [43]:
%%time
io_df_read_write_worst["index"].count().compute()

CPU times: user 1.46 s, sys: 63.5 ms, total: 1.52 s
Wall time: 2.33 s


0

In [29]:
%%time
tstart_task = io_df_read_write_worst["tstart"].min()
tend_task = io_df_read_write_worst["tend"].max()
res = dask.compute(tstart_task, tend_task)
res

CPU times: user 18.4 s, sys: 319 ms, total: 18.7 s
Wall time: 37.9 s


(6.499417304992676, 25.40230369567871)

In [30]:
tstart, tend = res
tbins, tbins_step = np.linspace(tstart, tend, num=n_bins, retstep=True)
tbins

array([ 6.4994173 ,  8.59973802, 10.70005873, 12.80037944, 14.90070015,
       17.00102086, 19.10134157, 21.20166228, 23.30198299, 25.4023037 ])

In [32]:
%%time

io_df_read_write_worst["tbin"] = 0

for tbin in tbins:
    print(tbin, tbin + tbins_step)
    tstart_cond = io_df_read_write_worst["tstart"].ge(tbin - 1)
    tend_cond = io_df_read_write_worst["tend"].lt(tbin + tbins_step)
    io_df_read_write_worst["tbin"] = io_df_read_write_worst["tbin"].mask(tstart_cond & tend_cond, tbin)

6.499417304992676 8.599738015068901
8.599738015068901 10.700058725145126
10.700058725145128 12.800379435221355
12.800379435221355 14.90070014529758
14.90070014529758 17.001020855373806
17.001020855373806 19.10134156545003
19.101341565450035 21.20166227552626
21.20166227552626 23.301982985602486
23.301982985602486 25.40230369567871
25.40230369567871 27.502624405754936
CPU times: user 235 ms, sys: 20.1 ms, total: 255 ms
Wall time: 247 ms


In [33]:
io_df_read_write_worst.head()

,index,rank,thread_id,cat,tstart,tend,func_id,level,arg_count,args_1,...,args_7,args_8,args_9,args_10,duration,filename,size,count,bandwidth,tbin
35,36,0,312784,0,18.552891,18.677382,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.124491,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,514.094471,19.101342
38,39,0,312784,0,18.677414,18.720806,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.043392,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1474.920088,19.101342
41,42,0,312784,0,18.720913,18.840509,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.119596,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,535.132801,19.101342
44,45,0,312784,0,18.840620,18.879042,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.038422,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1665.728356,19.101342
47,48,0,312784,0,18.879133,18.925989,write,0,3,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,...,,,,,0.046856,/p/gpfs1/iopp/temp/hacc_dir/test-Part00000000-...,67108864.0,1.0,1365.889115,19.101342


In [34]:
%%time

tbin_tasks = []

for tbin in tbins:
    tbin_binned = io_df_read_write_worst[io_df_read_write_worst["tbin"] == tbin]
    tbin_ct = tbin_binned["index"].count()
    tbin_tasks.append(tbin_ct)
    
tbin_task_results, = dask.compute(tbin_tasks)

tbin_task_results

CPU times: user 21.3 s, sys: 368 ms, total: 21.7 s
Wall time: 40.9 s


[39, 566, 2752, 2269, 2415, 2077, 3207, 4997, 5558, 390]

In [35]:
worst_tbin_task_results_ixs = np.argpartition(tbin_task_results, 5)[5:]
print(worst_tbin_task_results_ixs)
print([tbin_task_results[i] for i in worst_tbin_task_results_ixs])
worst_tbins = [tbins[i] for i in worst_tbin_task_results_ixs]
worst_tbins

[4 6 7 8 2]
[2415, 3207, 4997, 5558, 2752]


[14.90070014529758,
 19.101341565450035,
 21.20166227552626,
 23.301982985602486,
 10.700058725145128]

In [36]:
io_df_read_write_worst_2 = io_df_read_write_worst[io_df_read_write_worst["tbin"].isin(worst_tbins)]

In [37]:
%%time
io_df_read_write_worst_2["index"].count().compute()

18929

In [38]:
%%time
tstart_task = io_df_read_write_worst_2["tstart"].min()
tend_task = io_df_read_write_worst_2["tend"].max()
res = dask.compute(tstart_task, tend_task)
res

CPU times: user 19.5 s, sys: 324 ms, total: 19.8 s
Wall time: 39.1 s


(9.700082778930664, 25.068326950073242)

In [39]:
tstart, tend = res
tbins, tbins_step = np.linspace(tstart, tend, num=n_bins, retstep=True)
tbins

array([ 9.70008278, 11.40766546, 13.11524815, 14.82283084, 16.53041352,
       18.23799621, 19.94557889, 21.65316158, 23.36074426, 25.06832695])

In [40]:
%%time

io_df_read_write_worst_2["tbin"] = 0

for tbin in tbins:
    print(tbin, tbin + tbins_step)
    tstart_cond = io_df_read_write_worst_2["tstart"].ge(tbin - 1)
    tend_cond = io_df_read_write_worst_2["tend"].lt(tbin + tbins_step)
    io_df_read_write_worst_2["tbin"] = io_df_read_write_worst_2["tbin"].mask(tstart_cond & tend_cond, tbin)

9.700082778930664 11.407665464613173
11.407665464613173 13.115248150295681
13.115248150295681 14.82283083597819
14.822830835978191 16.5304135216607
16.530413521660698 18.23799620734321
18.23799620734321 19.94557889302572
19.94557889302572 21.65316157870823
21.653161578708225 23.360744264390735
23.360744264390732 25.068326950073242
25.068326950073242 26.775909635755752
CPU times: user 230 ms, sys: 19.8 ms, total: 250 ms
Wall time: 241 ms


In [41]:
%%time

tbin_tasks = []

for tbin in tbins:
    tbin_binned = io_df_read_write_worst_2[io_df_read_write_worst_2["tbin"] == tbin]
    tbin_ct = tbin_binned["index"].count()
    tbin_tasks.append(tbin_ct)
    
tbin_task_results, = dask.compute(tbin_tasks)

tbin_task_results

CPU times: user 21.7 s, sys: 256 ms, total: 22 s
Wall time: 41 s


[372, 2380, 0, 1814, 599, 1396, 3235, 3627, 5068, 436]

In [42]:
worst_tbin_task_results_ixs = np.argpartition(tbin_task_results, 5)[5:]
print(worst_tbin_task_results_ixs)
print([tbin_task_results[i] for i in worst_tbin_task_results_ixs])
worst_tbins = [tbins[i] for i in worst_tbin_task_results_ixs]
worst_tbins

[3 1 7 8 6]
[1814, 2380, 3627, 5068, 3235]


[14.822830835978191,
 11.407665464613173,
 21.653161578708225,
 23.360744264390732,
 19.94557889302572]

In [43]:
io_df_read_write_worst_3 = io_df_read_write_worst_2[io_df_read_write_worst_2["tbin"].isin(worst_tbins)]

In [44]:
%%time
io_df_read_write_worst_3["index"].count().compute()

CPU times: user 19.7 s, sys: 424 ms, total: 20.1 s
Wall time: 40.1 s


16124

In [50]:
min(worst_tbins)-1,max(worst_tbins)+tbins_step

(10.407665464613173, 25.068326950073242)

In [51]:
%%time
tstart_task = io_df_read_write_worst_3["tstart"].min()
tend_task = io_df_read_write_worst_3["tend"].max()
res = dask.compute(tstart_task, tend_task)
res

CPU times: user 19.9 s, sys: 409 ms, total: 20.3 s
Wall time: 40.6 s


(10.412095069885254, 24.68063735961914)

In [52]:
tstart, tend = res
tbins, tbins_step = np.linspace(tstart, tend, num=n_bins, retstep=True)
tbins

array([10.41209507, 11.99748866, 13.58288225, 15.16827583, 16.75366942,
       18.33906301, 19.9244566 , 21.50985018, 23.09524377, 24.68063736])

In [53]:
%%time

io_df_read_write_worst_3["tbin"] = 0

for tbin in tbins:
    print(tbin, tbin + tbins_step)
    tstart_cond = io_df_read_write_worst_3["tstart"].ge(tbin - 1)
    tend_cond = io_df_read_write_worst_3["tend"].lt(tbin + tbins_step)
    io_df_read_write_worst_3["tbin"] = io_df_read_write_worst_3["tbin"].mask(tstart_cond & tend_cond, tbin)

10.412095069885254 11.997488657633463
11.997488657633463 13.582882245381672
13.582882245381674 15.168275833129883
15.168275833129883 16.753669420878094
16.753669420878094 18.339063008626304
18.339063008626304 19.924456596374515
19.92445659637451 21.509850184122723
21.509850184122723 23.095243771870933
23.095243771870933 24.680637359619144
24.68063735961914 26.26603094736735
CPU times: user 258 ms, sys: 0 ns, total: 258 ms
Wall time: 249 ms


In [54]:
%%time

tbin_tasks = []

for tbin in tbins:
    tbin_binned = io_df_read_write_worst_3[io_df_read_write_worst_3["tbin"] == tbin]
    tbin_ct = tbin_binned["index"].count()
    tbin_tasks.append(tbin_ct)
    
tbin_task_results, = dask.compute(tbin_tasks)

tbin_task_results

CPU times: user 22.1 s, sys: 515 ms, total: 22.6 s
Wall time: 43.1 s


[487, 1888, 203, 1610, 0, 0, 2702, 3831, 4226, 1169]

In [55]:
worst_tbin_task_results_ixs = np.argpartition(tbin_task_results, 5)[5:]
print(worst_tbin_task_results_ixs)
print([tbin_task_results[i] for i in worst_tbin_task_results_ixs])
worst_tbins = [tbins[i] for i in worst_tbin_task_results_ixs]
worst_tbins

[3 1 7 8 6]
[1610, 1888, 3831, 4226, 2702]


[15.168275833129883,
 11.997488657633463,
 21.509850184122723,
 23.095243771870933,
 19.92445659637451]

In [56]:
io_df_read_write_worst_4 = io_df_read_write_worst_3[io_df_read_write_worst_3["tbin"].isin(worst_tbins)]

In [57]:
%%time
io_df_read_write_worst_3["index"].count().compute()

CPU times: user 20.2 s, sys: 384 ms, total: 20.6 s
Wall time: 42 s


16124